## Librerie e funzioni

In [19]:
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# Parametri geometrici delle lastre
larghezza_pla = 26.7  # cm
lunghezza_pla = 52.5  # cm
profondità_pla = 2.3  # cm

# Distanze tra i PMT
distanza_7_4 = 28.9  # cm
distanza_4_2 = 39.4  # cm
distanza_2_b = 7  # cm
distanza_b_1=6

# Efficienze dei PMT
efficienze = {
    "PMT7": 0.35,
    "PMT4": 0.729,
    "PMT2": 0.3,
    "PMT1": 0.15,
    "PMT8": 0.3,
    "PMT9": 0.4,
    "PMT10": 0.3,
    "PMT11": 0.4
}

# Dimensioni del bersaglio (PMT sotto il sistema principale)
larghezza_bers = 30  # cm
lunghezza_bers = 31  # cm

# Dimensioni di ciascun PMT nel bersaglio
larghezza_pmt_bers = larghezza_bers / 2  # ogni PMT largo metà del sistema
lunghezza_pmt_bers = lunghezza_bers
profondità_pmt_bers=30/2

offset_x=(lunghezza_pla-lunghezza_bers)/2
offset_y=(larghezza_bers-larghezza_pla)/2


## Toy MonteCarlo START

In [21]:
# Funzione per verificare se una posizione è all'interno dei limiti geometrici
def is_inside(x, y, lunghezza,larghezza):
    return (0<= x <= lunghezza) and (0 <= y <= larghezza)

"""
# Funzione per verificare se una posizione è all'interno dei limiti geometrici
def is_inside_target(x, y, lunghezza,larghezza):
    return (offset_x<= x <= lunghezza+offset_x) and (-offset_y <= y <= larghezza)
    """

# Funzione per verificare se una posizione è all'interno dei limiti geometrici considerando diverse condizioni degli scintillatori del bersaglio
def is_inside_target(pmt, x, y, lunghezza, larghezza):
    if pmt in ["PMT9", "PMT11"]:
        return (offset_x <= x <= lunghezza + offset_x) and (-offset_y <= y <= larghezza-offset_y)
    elif pmt in ["PMT8", "PMT10"]:
        return (offset_x <= x <= lunghezza + offset_x) and (larghezza-offset_y <= y <= 2*larghezza-offset_y)
    else:
        # Default logic
        return (offset_x <= x <= lunghezza + offset_x) and (-offset_y <= y <= larghezza-offset_y)
    
# Simulazione di un evento per i PMT bersaglio
def simulate_target_event(xb89, yb89, theta, phi):
    # Limiti del sistema bersaglio rispetto a PMT2    
    xb01 = xb89 - profondità_pmt_bers * np.tan(theta) * np.cos(phi)
    yb01 = yb89 - profondità_pmt_bers * np.tan(theta) * np.sin(phi)

    # Coordinate dei centri dei PMT bersaglio
    pmt_positions = {
        "PMT8": (xb89, yb89),
        "PMT9": (xb89, yb89),
        "PMT10": (xb01, yb01),
        "PMT11": (xb01, yb01), 
    }

    # Verifica hit su ciascun PMT
    hit_results = []
    for pmt, (px, py) in pmt_positions.items():
        is_hit = is_inside_target(pmt, px, py, lunghezza_pmt_bers,larghezza_pmt_bers) and np.random.uniform(0, 1) < efficienze[pmt]
        hit_results.append(is_hit)
    # Logica OR tra i PMT del bersaglio
    return any(hit_results)

# Simulazione di un evento principale (PMT superiori + bersaglio)
def simulate_event():
    # Generazione casuale di theta e phi
    phi = np.random.uniform(0, 2 * np.pi)  # Distribuzione uniforme in phi
    theta = np.arccos(np.random.uniform(0, 1)**(1/3)) # Distribuzione cos^2(theta)

    # PMT7
    x1 = np.random.uniform(0, 3*lunghezza_pla )
    y1 = np.random.uniform(0, 3*larghezza_pla)
    if not is_inside(x1, y1, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT7"]:
        return False

    # PMT4
    x2 = x1 - distanza_7_4 * np.tan(theta) * np.cos(phi)
    y2 = y1 - distanza_7_4 * np.tan(theta) * np.sin(phi)
    if not is_inside(x2, y2,lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT4"]:
        return False
    
    # PMT2
    x3 = x2 - distanza_4_2 * np.tan(theta) * np.cos(phi)
    y3 = y2 - distanza_4_2 * np.tan(theta) * np.sin(phi)
    if not is_inside(x3, y3, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT2"]:
        return False

    # Logica AND tra i PMT superiori e il bersaglio
    x89= x3 - distanza_2_b * np.tan(theta) * np.cos(phi)
    y89 = y3 - distanza_2_b* np.tan(theta) * np.sin(phi)
    return simulate_target_event(x89, y89,theta,phi)

# Simulazione Monte Carlo
def monte_carlo_simulation(duration, area):
    total_events = int(duration/60 * area)  # Numero totale di eventi da simulare
    coincident_events = 0
    for _ in range(total_events):
        if simulate_event():
            coincident_events += 1
    return coincident_events / duration  # Rate di coincidenza

# Esecuzione della simulazione
tempo_acquisizione = 1000 # secondi
area_totale = larghezza_pla * lunghezza_pla  # cm²
rate_coincidenza = monte_carlo_simulation(tempo_acquisizione, area_totale)
print(f"Rate di coincidenza  AND (sistema completo) per {tempo_acquisizione} s: {rate_coincidenza:.5f} eventi/s")

print(f"Quindi ho n eventi per ora {rate_coincidenza*3600:.5f} eventi")

Rate di coincidenza  AND (sistema completo) per 1000 s: 0.00500 eventi/s
Quindi ho n eventi per ora 18.00000 eventi


In [ ]:
# Simulazione di un evento principale
def simulate_event_veto():
    
    # Generatore di numeri casuali
    rng = np.random.default_rng() #random seed
    
    # Selezione casuale per energia dei muoni (Genera l'energia dei muoni basandosi su una distribuzione piatta per energie <1 GeV.)
    muon_energy = rng.uniform(0, 1) * 1000 #da GeV a MeV
    if muon_energy > 100:
        return False  # I muoni con energia > 150 MeV non raggiungono il bersaglio
    #else:
     #   print(muon_energy)
    
    # Generazione casuale di theta e phi
    phi = rng.uniform(0, 2 * np.pi)  # Distribuzione uniforme in phi
    theta = np.arccos(rng.uniform(0, 1)**(1/3)) # Distribuzione cos^2(theta)

    # PMT7
    x1 = rng.uniform(0, 3*lunghezza_pla )
    y1 = rng.uniform(0, 3*larghezza_pla)
    if not is_inside(x1, y1, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT7"]:
        return False

    # PMT4
    x2 = x1 - distanza_7_4 * np.tan(theta) * np.cos(phi)
    y2 = y1 - distanza_7_4 * np.tan(theta) * np.sin(phi)
    if not is_inside(x2, y2,lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT4"]:
        return False
    
    # PMT2
    x3 = x2 - distanza_4_2 * np.tan(theta) * np.cos(phi)
    y3 = y2 - distanza_4_2 * np.tan(theta) * np.sin(phi)
    if not is_inside(x3, y3, lunghezza_pla,larghezza_pla) or np.random.uniform(0, 1) > efficienze["PMT2"]:
        return False

    # Logica AND tra i PMT superiori e il bersaglio
    x89= x3 - distanza_2_b * np.tan(theta) * np.cos(phi)
    y89 = y3 - distanza_2_b* np.tan(theta) * np.sin(phi)

    if not simulate_target_event(x89, y89,theta,phi):
         return False

    xb01 = x89- profondità_pmt_bers * np.tan(theta) * np.cos(phi)
    yb01 = y89 - profondità_pmt_bers * np.tan(theta) * np.sin(phi)
    
    # PMT1
    x4= xb01-distanza_b_1*np.tan(theta)*np.cos(phi)
    y4= yb01-distanza_b_1*np.tan(theta)*np.cos(phi)

    if is_inside(x4, y4, lunghezza_pla,larghezza_pla) and np.random.uniform(0, 1) < efficienze["PMT1"]:
         return False
    
    return True

# Simulazione Monte Carlo
def monte_carlo_simulation_and_veto(duration, area):
    total_events = int(duration/60 * area)  # Numero totale di eventi da simulare, con intensità di particelle di 1 particella per minuto su cm^2
    coincident_events = 0
    for _ in range(total_events):
        if simulate_event_veto():
            coincident_events += 1
    print(f"Totale coincidenze simulate per {coincident_events}")
    return coincident_events / duration  # Rate di coincidenza

# Esecuzione della simulazione
tempo_acquisizione = 10000 # secondi
area_totale = larghezza_pla * lunghezza_pla  # cm²
rate_coincidenza = monte_carlo_simulation_and_veto(tempo_acquisizione, area_totale)
print(f"Rate di coincidenza con VETO (sistema completo) per {tempo_acquisizione} s: {rate_coincidenza:.5f} eventi/s")

print(f"Quindi ho n eventi per ora {rate_coincidenza*3600:.5f} eventi")

31.035348482586823
57.75175109231556
67.53860672055356
22.750430571692238
44.05831902603208
74.51644482574748
56.17031337906098
1.1472221721742226
0.3093362462873728
57.55532632853366
27.00366097441742
71.48810735778622
15.571261314601736
30.524916721724193
7.444249591578189
64.83016325915014
44.81996231647856
84.89589384406126
15.1708750497469
55.95866446204767
62.854317026836966
99.94730939631502
2.3331038927374426
23.103383006071844
80.7473332541776
26.760312322324566
42.294892562281476
60.16396077565744
32.38105880722553
30.62692734521011
31.033561689848387
50.47771651172261
41.60142162481695
12.965526388936222
23.28235030990289
66.59342344191133
84.68222968999561
74.7850039445227
82.9630469794559
95.28424662999258
12.90270223401202
42.57500320458429
73.86771981155094
70.42889514995987
10.917314174982096
4.115439257648901
24.379937113539143
20.037936394479814
41.365347999037574
38.600770794237626
83.44448793690573
42.013879754678406
2.970616414380789
60.28707979976433
70.3288755918

## Toy MonteCarlo STOP

## Efficienza intrinseca del VETO

In [55]:
def errore_prodotto(x,y,dx,dy):
    return np.sqrt((y*dx)**2+(x*dy)**2)
def errore_prodotto3(x,dx, y,dy, z, dz):
    return np.sqrt((x*y*dz)**2+(x*z*dy)**2+(dx*y*z)**2)

In [56]:
def accettanza_sistema3(length1, length2, length3, width,height1, height2):
    """Calcolo della efficienza geometrica per 3 scintillatori sovrapposti con geometria diversa.
    L'altezza è positiva se considerata dal basso verso l'alto, negativa se viceversa.
    1 e 2 rappresentano la doppia, mentre 3 rappresenta lo scintillatore di cui vogliamo calcolare l'accettanza geometrica."""
    n = 10000 #Numero di tentativi
    # Inizializza contatori
    successi = 0
    tentativi = 0
    
    # Generatore di numeri casuali
    rng = np.random.default_rng() #random seed
    
    for _ in range(n):
        # Genera coordinate x1 e y1 sul primo scintillatore
        x1 = rng.uniform(0, width)
        y1 = rng.uniform(0, length1)
        
        
        # Genera angoli ϕ e θ
        phi = rng.uniform(0, 2 * np.pi)
        theta = np.arccos(rng.uniform(0, 1)**(1/4))  # distribuito come cos3(θ)
        
        # Proiezione del fascio sul secondo scintillatore
        x2 = x1 - height1 * np.tan(theta) * np.cos(phi) 
        y2 = y1 - height1 * np.tan(theta) * np.sin(phi)

        # Proiezione del fascio sul terzo scintillatore
        x3 = x2 - height2 * np.tan(theta) * np.cos(phi) 
        y3 = y2 - height2 * np.tan(theta) * np.sin(phi)
        
        # Verifica se x2 e y2 sono compresi nelle dimensioni del secondo scintillatore
        if 0 <= x2 <= width and 0 <= y2 <= length2:
            if 0 <= x3 <= width and 0 <= y3 <= length3:
                successi += 1
            tentativi+=1 #probabilità che passi dal terzo PMT data la doppia
    
    efficienza_geometrica = successi / tentativi
    errore = np.sqrt(efficienza_geometrica * (1 - efficienza_geometrica) / tentativi)
    return efficienza_geometrica, errore

In [ ]:
larghezza_pla=26.7 #cm
dlarghezza_pla=0.1#cm
lunghezza_pla=52.5 
dlunghezza_pla=0.1
prodondità_pla=2.3
dprofondità_pla=0.1

area_pla=larghezza_pla*lunghezza_pla
darea=errore_prodotto(lunghezza_pla,dlunghezza_pla,larghezza_pla,dlarghezza_pla)

distanza_7_4=28.9 #cm
distanza_4_3=9.4 
distanza_3_2=9.4
distanza_2_b=7
distanza_b_1=15

In [ ]:
triple_doppie1=0.13
errore1=0.03
triple_doppie2=0.32
errore2=0.03
triple_doppie4=0.73
errore4=0.04
triple_doppie7=0.34
errore7=0.03

distanza_2_b+distanza_b_1
acc_1, errore_acc_1=accettanza_sistema3(lunghezza_pla,lunghezza_pla,lunghezza_pla, larghezza_pla, distanza_7_4+distanza_2_b)
print(f'Accettanza1: {acc_1}')

epsilon_intr1=triple_doppie1/acc_1
depsilon_intr1=errore_prodotto(triple_doppie1,acc_1,errore1,errore_acc_1)
print(f"Efficienza intrinseca PMT1 {epsilon_intr1:.2f} ± {depsilon_intr1:.2f}")

NameError: name 'accettanza_sistema3' is not defined

In [9]:
geom_eff1, err_geom1=accettanza_sistema2(length2,length1,width,height1)
print(f"Efficienza geometrica 1: {geom_eff1:.3f}  ± {err_geom1:.3f}")

geom_eff2, err_geom2=accettanza_sistema2(length1,length2,width,-height1)
print(f"Efficienza geometrica 2: {geom_eff2:.3f}  ± {err_geom2:.3f}")

epsilon_eff1=epsilon_intr1*geom_eff1
depsilon_eff1=errore_prodotto(epsilon_intr1,geom_eff1,depsilon_intr1,err_geom1)
print(f"Efficienza corretta per l'accettanza 1: {epsilon_eff1:.2f} ± {depsilon_eff1:.2f}")

epsilon_eff2=epsilon_intr2*geom_eff2
depsilon_eff2=errore_prodotto(epsilon_intr2,geom_eff2,depsilon_intr2,err_geom2)
print(f"Efficienza corretta per l'accettanza 2: {epsilon_eff2:.2f} ± {depsilon_eff2:.2f}")

Efficienza geometrica 1: 0.938  ± 0.002
Efficienza geometrica 2: 0.605  ± 0.005
Efficienza corretta per l'accettanza 1: 0.89 ± 0.05
Efficienza corretta per l'accettanza 2: 0.53 ± 0.03


In [10]:
epsilon_eff=epsilon_eff1*epsilon_eff2
depsilon_eff=errore_prodotto(epsilon_eff1,depsilon_eff1,epsilon_eff2,depsilon_eff2)
print(f"Efficienza Setup08: {epsilon_eff:.2f} ± {depsilon_eff:.2f}")

Efficienza Setup08: 0.47 ± 0.04


In [14]:
rate_cosmic1=cosmic*aeff*1/60
drate_cosmic1=cosmic*daeff*1/60
print(f"Rate Setup08: {epsilon_eff*rate_cosmic1} ± {errore_prodotto(rate_cosmic1, epsilon_eff, drate_cosmic1, depsilon_eff)}")

Rate Setup08: 6.081140520476354 ± 0.5236151502970052


In [17]:
rate1=4548
drate1=12
rate2=1063
drate2=6
racc=rate1*rate2*2*(40e-9-2e-9)
dracc=np.sqrt((drate1*rate2)**2+(rate1*drate2)**2)*2*(40e-9-2e-9)
print(f"Rate eventi accidentali: {racc*1e3:.0f}± {dracc*1e3:.0f} mHz")

Rate eventi accidentali: 367± 2 mHz
